## Objective

Build a recommendation system that combines weather data and local events to suggest activities, demonstrating practical multi-agent coordination. This include:

1.   The system generates clear, context-aware recommendations using GPT.
2.   Weather data and event types are both meaningfully considered in suggestions.


In [1]:
import sqlite3


"""" The function setup_database() sets up the database connection using sqlite3 and
creates database events.db containing the table events that stores events information.
Events information includes the columns name, type, description, location, date.
"""


def setup_database():
    conn = sqlite3.connect('events.db')
    c = conn.cursor()

    c.execute('''
        CREATE TABLE IF NOT EXISTS events (
            id INTEGER PRIMARY KEY,
            name TEXT,
            type TEXT,  -- 'indoor' or 'outdoor'
            description TEXT,
            location TEXT,
            date TEXT
        )
    ''')

    # Sample events
    events = [
        ('Summer Concert', 'outdoor', 'Live music in the park', 'Central Park', '2025-07-15'),
        ('Art Exhibition', 'indoor', 'Modern art showcase', 'City Gallery', '2025-07-15'),
        ('Food Festival', 'outdoor', 'International cuisine', 'Waterfront', '2025-07-16'),
        ('Theater Show', 'indoor', 'Classical drama', 'Grand Theater', '2025-07-16')
    ]

    c.executemany('INSERT OR IGNORE INTO events (name, type, description, location, date) VALUES (?,?,?,?,?)', events)
    conn.commit()
    conn.close()


In [7]:
import openai
import requests
import sqlite3

from google.colab import userdata
weather_api_key = userdata.get('WEATHER_API_KEY')
openai_api_key = userdata.get('OPENAI_API_KEY')

"""
The class WeatherAgent gets the current weather using Weather API .
Exception handling is included to handle issues during code execution.
"""


class WeatherAgent:
    def __init__(self, api_key):
        self.api_key = api_key

    def get_weather(self, location, date):
        # Use current weather since we're dealing with same-day recommendations
        url = f"http://api.weatherapi.com/v1/current.json"
        params = {
            "key": self.api_key,
            "q": location,
            "aqi": "no"
        }
        try:
            response = requests.get(url, params=params)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            raise Exception(f"Weather API error: {str(e)}")


"""
EventAgent class queries the events.db database and fetches events data based on certain filter
conditions like date and event type.
"""


class EventAgent:
    def get_events(self, date, event_type=None):
        #print("event_type", event_type)
        conn = sqlite3.connect('events.db')
        c = conn.cursor()

        try:
            if event_type:
                c.execute('SELECT * FROM events WHERE date = ? AND type = ?', (date, event_type))
            else:
                c.execute('SELECT * FROM events WHERE date = ?', (date,))

            events = c.fetchall()
            return events
        except sqlite3.Error as e:
            raise Exception(f"Database error: {str(e)}")
        finally:
            conn.close()


"""
The class RecommendationAgent generates events recommendation based on the weather.
It also creates context for GPT, calls the GPT-4 model.

"""


class RecommendationAgent:
    def __init__(self, openai_api_key):
        self.api_key = openai_api_key
        self.client = openai.OpenAI(api_key=openai_api_key)

    def generate_recommendation(self, weather_data, events):
        # Create context for GPT
        try:
            # Handle both current and forecast data
            if 'current' in weather_data:
                weather_condition = weather_data['current']['condition']['text']
                temperature = weather_data['current']['temp_c']
                context = f"Weather: {weather_condition}, Temperature: {temperature}°C\n\n"
            else:
                context = "Weather data unavailable\n\n"

            context += "Available events:\n"
            for event in events:
                context += f"- {event[1]} ({event[2]}): {event[3]} at {event[4]}\n"

            response = self.client.chat.completions.create(
                model="gpt-4",
                messages=[
                    {"role": "system", "content": """You are a helpful event recommender. Consider the weather conditions
                    and suggest suitable events. For outdoor events, consider the temperature and weather conditions.
                    Be specific about why you recommend certain events over others. Keep your response concise but informative.
                    If weather data is unavailable, focus on providing a balanced recommendation of both indoor and outdoor events."""},
                    {"role": "user", "content": context}
                ]
            )
            return response.choices[0].message.content
        except Exception as e:
            raise Exception(f"Recommendation error: {str(e)}")


"""
The class CoordinatorAgent acts as an orchestrator/planner that controls weather, event and recommendation agents.
It gets weather data, events data and calls recommendation_agent to display suggestions based on the weather
and events available. The overall execution of the program starts from CoordinatorAgent class.
"""


class CoordinatorAgent:
    def __init__(self, weather_api_key, openai_api_key):
        self.weather_agent = WeatherAgent(weather_api_key)
        self.event_agent = EventAgent()
        self.recommendation_agent = RecommendationAgent(openai_api_key)

    def get_recommendations(self, location, date):
        try:
            # Get weather data
            print(f"\nFetching weather data for {location} on {date}...")
            weather_data = self.weather_agent.get_weather(location, date)

            # Get events
            print("Fetching events...")
            events = self.event_agent.get_events(date)

            if not events:
                return "No events found for this date."

            # Generate recommendations
            print("Generating recommendations...")
            recommendations = self.recommendation_agent.generate_recommendation(
                weather_data, events
            )

            return recommendations

        except Exception as e:
            return f"Error: {str(e)}"


if __name__ == "__main__":
    setup_database()
    # Test dates
    test_cases = [
        ("2025-07-15", "Singapore"),
        ("2025-07-16", "Singapore"),
        ("2025-07-17", "Singapore"),  # No events on this date
    ]

    coordinator = CoordinatorAgent(weather_api_key, openai_api_key)

    for test_date, test_location in test_cases:
        print(f"\n{'=' * 50}")
        print(f"Getting recommendations for {test_location} on {test_date}:")
        print(f"{'=' * 50}")
        print(coordinator.get_recommendations(test_location, test_date))


Getting recommendations for Singapore on 2025-07-15:

Fetching weather data for Singapore on 2025-07-15...
Fetching events...
Generating recommendations...
Considering the warm temperature of 31.0°C with partly cloudy conditions, both the outdoor Summer Concert at Central Park and the indoor Art Exhibition at City Gallery would be suitable events.

If you enjoy outdoor activities and live music, you might find the Summer Concert appealing. There's enough cloud cover to provide some shade from the sun, making it a comfortable setting to enjoy some live music.

However, if the heat is a bit much for you, you might prefer the Art Exhibition. It's indoors, so you'd be able to escape the heat and enjoy some fine modern art at the same time.

I would recommend both events because they cater to different preferences and both weather conditions are suitable for each event. However, make sure to keep yourself hydrated if you choose to attend the summer concert considering the high temperature.